In [1]:
!pip install numpy matplotlib tensorflow

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
pip install kaggle


Note: you may need to restart the kernel to use updated packages.


In [4]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

Dataset URL: https://www.kaggle.com/datasets/anandkumarsahu09/cattle-breeds-dataset
License(s): CC0-1.0
cattle-breeds-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!tar -xf cattle-breeds-dataset.zip


In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=10,       # rotate images randomly
    width_shift_range=0.05,   # shift horizontally
    height_shift_range=0.05,  # shift vertically
    zoom_range=0.1,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [7]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [8]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True


In [9]:
for layer in base_model.layers[:100]:
    layer.trainable = False


In [10]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])


In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(           #If validation loss does not improve for 5 consecutive epochs, stop training automatically
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(       #If validation loss stops improving, reduce the learning rate.”
        monitor="val_loss",
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.4654 - loss: 1.3679 - val_accuracy: 0.5523 - val_loss: 1.1771 - learning_rate: 1.0000e-04
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 529ms/step - accuracy: 0.7564 - loss: 0.6985 - val_accuracy: 0.6067 - val_loss: 1.1578 - learning_rate: 1.0000e-04
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 559ms/step - accuracy: 0.8452 - loss: 0.4378 - val_accuracy: 0.6444 - val_loss: 1.0439 - learning_rate: 1.0000e-04
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 537ms/step - accuracy: 0.9092 - loss: 0.3065 - val_accuracy: 0.6485 - val_loss: 1.0183 - learning_rate: 1.0000e-04
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 556ms/step - accuracy: 0.9412 - loss: 0.1956 - val_accuracy: 0.6151 - val_loss: 1.1288 - learning_rate: 1.0000e-04
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 549ms/step - accuracy: 0.9618 - loss: 0.1571 - val_accuracy: 0.6736 - val_loss: 0.9895 - learning_rate: 1.0000e-04
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 530ms/step - accura

In [13]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy after fine-tuning: {val_acc*100:.2f}%")


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 300ms/step - accuracy: 0.6946 - loss: 0.9574
Validation Accuracy after fine-tuning: 69.46%


In [14]:
y_true = val_data.classes
y_true


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int32)

In [15]:
y_pred_probs = model.predict(val_data)
y_pred_probs


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 404ms/step


array([[6.0555182e-04, 9.4985086e-01, 4.6434584e-03, 2.3154268e-02,
        2.1745855e-02],
       [1.4633254e-02, 9.1293230e-05, 2.0295289e-05, 2.2219082e-03,
        9.8303336e-01],
       [3.2682511e-01, 8.0135800e-03, 5.6801605e-01, 8.5372835e-02,
        1.1772331e-02],
       ...,
       [1.6883256e-02, 3.4334987e-01, 3.1037296e-03, 8.0189817e-02,
        5.5647337e-01],
       [9.8214543e-01, 4.4780163e-04, 1.3902782e-02, 1.4014618e-03,
        2.1025855e-03],
       [5.2546746e-01, 1.0004701e-03, 2.4589556e-04, 1.6010454e-03,
        4.7168508e-01]], shape=(239, 5), dtype=float32)

In [16]:
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)
y_pred


array([1, 4, 2, 1, 2, 2, 4, 2, 0, 2, 0, 4, 2, 2, 1, 2, 3, 2, 0, 4, 2, 0,
       3, 4, 1, 1, 2, 3, 4, 2, 2, 1, 3, 1, 2, 1, 2, 1, 2, 1, 4, 3, 4, 1,
       4, 3, 1, 4, 1, 3, 3, 0, 4, 1, 4, 2, 2, 1, 3, 2, 3, 4, 0, 0, 1, 2,
       3, 1, 1, 4, 0, 4, 3, 1, 4, 4, 2, 3, 0, 1, 4, 0, 1, 2, 4, 4, 3, 2,
       2, 1, 3, 0, 4, 2, 1, 4, 2, 1, 1, 2, 3, 3, 0, 0, 2, 4, 1, 4, 1, 3,
       1, 4, 4, 4, 2, 1, 2, 2, 4, 0, 2, 0, 2, 4, 3, 3, 3, 4, 4, 4, 0, 2,
       4, 2, 0, 1, 0, 1, 2, 4, 1, 0, 4, 1, 4, 3, 3, 4, 4, 4, 2, 3, 4, 2,
       4, 1, 1, 2, 4, 3, 3, 3, 1, 0, 3, 4, 4, 4, 4, 2, 4, 4, 1, 1, 4, 3,
       3, 1, 2, 2, 4, 0, 0, 0, 3, 1, 4, 4, 3, 1, 3, 4, 2, 0, 3, 2, 3, 2,
       3, 4, 0, 2, 1, 2, 3, 2, 3, 0, 0, 0, 4, 0, 1, 4, 2, 0, 3, 4, 4, 4,
       4, 2, 1, 0, 1, 0, 2, 0, 4, 3, 1, 1, 3, 3, 2, 2, 4, 0, 0])

In [17]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


[[ 5 13 16  8 10]
 [ 6 12 10  7 12]
 [ 8  8 10  8 16]
 [ 5  8  9 13 15]
 [11  6  8  6  9]]


In [18]:
print(val_data.class_indices)


{'Ayrshire cattle': 0, 'Brown Swiss cattle': 1, 'Holstein Friesian cattle': 2, 'Jersey cattle': 3, 'Red Dane cattle': 4}


In [19]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_data.class_indices.keys())
))


                          precision    recall  f1-score   support

         Ayrshire cattle       0.14      0.10      0.11        52
      Brown Swiss cattle       0.26      0.26      0.26        47
Holstein Friesian cattle       0.19      0.20      0.19        50
           Jersey cattle       0.31      0.26      0.28        50
         Red Dane cattle       0.15      0.23      0.18        40

                accuracy                           0.21       239
               macro avg       0.21      0.21      0.20       239
            weighted avg       0.21      0.21      0.20       239



In [20]:
model.save("cattle_breed_model.keras")
